In [16]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle

# Check if files exist
model_path = 'model.h5'
gender_encoder_path = 'lable_encoder_gender.pkl'
geo_encoder_path = 'ohe_geo.pkl'
scaler_path = 'scaler.pkl'

In [17]:
model = load_model(model_path)
    
with open(gender_encoder_path, 'rb') as file:
    label_encoder_gender = pickle.load(file)
        
with open(geo_encoder_path, 'rb') as file:
    ohe_geo = pickle.load(file)
        
with open(scaler_path, 'rb') as file:
    scaler = pickle.load(file)
        
   

In [18]:
sample_input = {
    'CreditScore':619,
    'Geography':'France',
    'Gender':'Female',
    'Age':42,
    'Tenure':2,
    'Balance':0,
    'NumOfProducts':1,
    'HasCrCard':1,
    'IsActiveMember':1,
    'EstimatedSalary':101348.88
}

In [19]:
#ohe


In [20]:
geo_encoded = ohe_geo.transform([[sample_input['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=ohe_geo.get_feature_names_out(['Geography']))
geo_encoded_df

D:\GeN AI complete\ann project\.venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [21]:
input_df=pd.DataFrame([sample_input])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,France,Female,42,2,0,1,1,1,101348.88


In [22]:
#encode categorical variable


In [23]:
input_df['Gender']=label_encoder_gender.transform([sample_input['Gender']])
input_df


,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,France,0,42,2,0,1,1,1,101348.88


In [24]:
input_df=pd.concat([input_df.drop("Geography", axis=1), geo_encoded_df], axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0,1,1,1,101348.88,1.0,0.0,0.0


In [25]:
#scalling the input dataa

In [27]:
input_scaled=scaler.transform(input_df)
input_scaled

array([[-0.33880827, -1.09499335,  0.29493847, -1.04241787, -1.21847056,
        -0.91668767,  0.64920267,  0.97481699,  0.01595384,  1.00150113,
        -0.57946723, -0.57638802]])

In [28]:
#prediction

In [36]:
prediction=model.predict(input_scaled)
prediction


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step


array([[0.13806678]], dtype=float32)

In [37]:
prediction_proba=prediction[0][0]

In [38]:
prediction_proba

np.float32(0.13806678)

In [39]:
if prediction_proba>0.5:
    print("customer likely to churn")
else:
    print("customer not likely to churn")

customer not likely to churn
